<a href="https://colab.research.google.com/github/amitshmidov/geometric_learning_project/blob/main/geometric_learning_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [3]:
import numpy as np
import tensorflow as tf
import pyvista as pv
import matplotlib.pyplot as plt
import os
from tqdm.tqdm_notebook import tqdm_notebook as tqdm

from typing import Tuple
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Utils

In [4]:
def read_off(off_file_path: str) -> Tuple[np.ndarray, np.ndarray]:
    """
    Read .off files to shared vertex data structure.
    Parameters
    ----------
    off_file_path: str
        Path to .off file

    Returns
    -------
    (v, f): tuple
        vertices and faces in shared vertex data structure.
    """
    with open(off_file_path, 'r') as off_file:
        off_data = off_file.read().split('\n')
    meta_nums = off_data[off_data[0] == 'OFF'].split()
    v_num, f_num = int(meta_nums[0]), int(meta_nums[1])
    off_data = off_data[1 + (off_data[0] == 'OFF'):]

    v = np.array(list(map(lambda s: list(map(lambda c: float(c), s.split()[:3])), off_data[:v_num])))
    f = np.array(list(map(lambda s: list(map(lambda c: int(c), s.split()[1:4])), off_data[v_num:v_num + f_num])))
    return v, f

# Data loading

In [15]:
v, f = read_off('/content/drive/My Drive/Technion/Master/Geometric Learning/Final Project/ModelNet10/ModelNet10/bed/train/bed_0001.off')
f_with_dim = np.hstack((3 * np.ones(f.shape[0]).astype(int)[:, np.newaxis], f))

dir_path = '/content/drive/My Drive/Technion/Master/Geometric Learning/Final Project/ModelNet10/ModelNet10'
classes = os.listdir(dir_path)
train_labels, test_labels = [], []
train_data, test_data = [], []
for c in tqdm(classes):
  # Train data loading:
  for mesh in os.listdir(dir_path + c + '/train')
    train_labels += [c]
    train_data += [read_off(dir_path + c + '/train' + mesh)[0]]

  # Test data loading:
  for mesh in os.listdir(dir_path + c + '/test')
    test_labels += [c]
    test_data += [read_off(dir_path + c + '/test' + mesh)[0]]


['bathtub',
 'bed',
 'chair',
 'desk',
 'dresser',
 'monitor',
 'night_stand',
 'sofa',
 'table',
 'toilet']

# Run basic experiments

## Classic **PointNet**

## Classic **Momenet**


In [6]:
# Constants:
BATCH_SIZE = 32
NUN_POINT = 1024
MAX_EPOCH = 250
BASE_LEARNING_RATE = 0.001
GPU_INDEX = 0
MOMENTUM = 0.9
OPTIMIZER = 'adam'
DECAY_STEP = 20000
DECAY_RATE = 0.8

MAX_NUM_POINT = 2048
NUN_CLASSES = 40

BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99

In [8]:
def add_moments(data, order=2):
  if order == 1:
    return data
  elif order == 2:
    data_moments = np.zeros((np.shape(data)[0], np.shape(data)[1], 9))
  elif order == 3:
    raise ValueError('3rd order moments are not prepared yet!')
  
  data_moments[:, :, 0:3] = data
  data_moments[:, :, 3] = data_moments[:, :, 0] * data_moments[:, :, 0]
  data_moments[:, :, 4] = data_moments[:, :, 1] * data_moments[:, :, 1]
  data_moments[:, :, 5] = data_moments[:, :, 2] * data_moments[:, :, 2]
  data_moments[:, :, 6] = data_moments[:, :, 0] * data_moments[:, :, 1]
  data_moments[:, :, 7] = data_moments[:, :, 0] * data_moments[:, :, 2]
  data_moments[:, :, 8] = data_moments[:, :, 1] * data_moments[:, :, 2]

  return data_moments

In [9]:
add_moments(v)

ValueError: ignored

In [ ]:
with tf.Graph().as_default():
    pointclouds_pl, labels_pl = MODEL.placeholder_inputs(BATCH_SIZE, NUM_POINT)
    is_training_pl = tf.placeholder(tf.bool, shape=())
    print(is_training_pl)
    
    # Note the global_step=batch parameter to minimize. 
    # That tells the optimizer to helpfully increment the 'batch' parameter for you every time it trains.
    batch = tf.Variable(0)
    bn_decay = get_bn_decay(batch)
    tf.summary.scalar('bn_decay', bn_decay)

    # Get model and loss 
    pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl, bn_decay=bn_decay)
    loss = MODEL.get_loss(pred, labels_pl, end_points)
    tf.summary.scalar('loss', loss)

    correct = tf.equal(tf.argmax(pred, 1), tf.to_int64(labels_pl))
    accuracy = tf.reduce_sum(tf.cast(correct, tf.float32)) / float(BATCH_SIZE)
    tf.summary.scalar('accuracy', accuracy)

    # Get training operator
    learning_rate = get_learning_rate(batch)
    tf.summary.scalar('learning_rate', learning_rate)
    if OPTIMIZER == 'momentum':
        optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=MOMENTUM)
    elif OPTIMIZER == 'adam':
        optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op = optimizer.minimize(loss, global_step=batch)
    
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()
        
    # Create a session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.allow_soft_placement = True
    config.log_device_placement = False
    sess = tf.Session(config=config)

    # Add summary writers
    #merged = tf.merge_all_summaries()
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'train'),
                              sess.graph)
    test_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'test'))

    # Init variables
    init = tf.global_variables_initializer()
    # To fix the bug introduced in TF 0.12.1 as in
    # http://stackoverflow.com/questions/41543774/invalidargumenterror-for-tensor-bool-tensorflow-0-12-1
    #sess.run(init)
    sess.run(init, {is_training_pl: True})

    ops = {'pointclouds_pl': pointclouds_pl,
            'labels_pl': labels_pl,
            'is_training_pl': is_training_pl,
            'pred': pred,
            'loss': loss,
            'train_op': train_op,
            'merged': merged,
            'step': batch}
    best_acc=0
    for epoch in range(MAX_EPOCH):
        log_string('**** EPOCH %03d ****' % (epoch))
        sys.stdout.flush()
          
        train_one_epoch(sess, ops, train_writer)
        acc = eval_one_epoch(sess, ops, test_writer)
        
        if acc > best_acc:
            save_path = saver.save(sess, os.path.join(LOG_DIR, "best_model.ckpt"))
            log_string("Best Model saved in file: %s" % save_path)
            best_acc = acc
        
        # Save the variables to disk.
        if epoch % 10 == 0:
            save_path = saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"))
            log_string("Model saved in file: %s" % save_path)

## $1^{st}$, $2^{nd}$ and $3^{rd}$ order moments

# Add consistently oriented vertex normals

## Classic **PointNet**

## Classic **Momenet**

## $1^{st}$, $2^{nd}$ and $3^{rd}$ order moments

# Add another geometric prelifting

## Basic runs

## Consistently oriented vertex normals runs